<a href="https://colab.research.google.com/github/UOS-COMP6252/public/blob/main/WGAN/wgan_gp_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Wassertein GAN

This is a notebook to generate images from Wasserstein GAN with gradient penalty pre-trained on the celebA dataset

In [ ]:
!pip install comet_ml

In [ ]:
%%sh
wget https://github.com/UOS-COMP6252/public/raw/main/WGAN/dcgan.py
wget https://github.com/UOS-COMP6252/public/raw/main/WGAN/utils.py

The pre-trained weights are stored in a comet_ml registry. You need an api key to access it

In [ ]:
import comet_ml
from comet_ml.integration.pytorch import load_model
import torch
from torchvision.utils import make_grid
from dcgan import Generator, Discriminator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

```comet_ml.init()``` will prompt for the api key

In [ ]:
comet_ml.init()

In [ ]:


# # The networks are taken from
# # https://arxiv.org/abs/1511.06434
# class TBlock(nn.Module):
#     def __init__(self, in_ch, out_ch, kernel_size,stride,pad,norm_type):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.ConvTranspose2d(in_ch,out_ch,kernel_size,stride, pad,bias=False),
#             norm_layer(out_ch,norm_type),
#             nn.ReLU()
#         )
#     def forward(self,x):
#         return self.net(x)
# class CBlock(nn.Module):
#     def __init__(self, in_ch, out_ch, kernel_size,stride,pad,norm_type: str = "batch"):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Conv2d(in_ch,out_ch,kernel_size,stride, pad, bias=False),
#             norm_layer(out_ch,norm_type),
#             nn.LeakyReLU(0.2)
#         )
#     def forward(self,x):
#         return self.net(x)

# class Generator(nn.Module):
#     #Outputs 64x64 pixel images

#     def __init__(
#         self,img_size=64,
#         out_ch=3,zdim=100,norm_type="BatchNorm2d",final_activation=None
#     ):
#         super().__init__()
#         # self.nf_g = nf_g
#         # self.z_dim = z_dim
#         # self.out_ch = out_ch
#         nf_g=2*img_size
#         self.final_activation=None if final_activation is None else getattr(torch,final_activation)

#         self.net = nn.Sequential(
#             # * Layer 1: 1x1
#             TBlock(zdim,8*nf_g, 4,1, 0,norm_type),
#             # * Layer 2: 4x4
#             TBlock(8*nf_g,4*nf_g,4,2,1,norm_type),
#             # * Layer 3: 8x8
#             TBlock(4*nf_g,2*nf_g,4,2,1,norm_type),
#             # * Layer 4: 16x16
#             TBlock(2*nf_g,nf_g,4,2,1,norm_type),
#             # * Layer 5: 32x32
#             nn.ConvTranspose2d(nf_g, out_ch, 4, 2, 1, bias=False),
#             # * Output: 64x64
#         )

#     def forward(self, x):
#         x = self.net(x)
#         return x if self.final_activation is None else self.final_activation(x)

#         #return torch.tanh(x)


# class Discriminator(nn.Module):
#     def __init__(self, img_size=64,in_ch=3,norm_type="BatchNorm2d",final_activation=None):
#         super().__init__()
#         nf_d=img_size
#         self.final_activation=None if final_activation is None else getattr(torch,final_activation)
#         self.net = nn.Sequential(
#             # * 64x64
#             nn.Conv2d(in_ch, nf_d, 4, 2, 1, bias=False),
#             nn.LeakyReLU(0.2),
#             # * 32x32
#             CBlock(nf_d,2*nf_d,4,2,1,norm_type),
#             # * 16x16
#             CBlock(2*nf_d,4*nf_d,4,2,1,norm_type),
#             # * 8x8
#             CBlock(4*nf_d,8*nf_d,4,2,1,norm_type),
#             # * 4x4
#             nn.Conv2d(8*nf_d, 1, 4, 1, 0, bias=False),
#         )


#     def forward(self, x):
#           x = self.net(x)
#           return x if self.final_activation is None else self.final_activation(x)

# class norm_layer(nn.Module):
#     def __init__(self, num_channels,norm_type: str = None):
#         super().__init__()
#         if norm_type == "BatchNorm2d":
#             self.norm = nn.BatchNorm2d(num_channels)
#         elif norm_type == "GroupNorm":
#             self.norm = nn.GroupNorm(num_channels, num_channels)
#         elif norm_type is None or norm_type == "None":
#             self.norm=None
#         else:
#             raise ValueError(f"Unknown normalization type: {norm_type}")

#     def forward(self, x):
#         return x if self.norm is None else self.norm(x)


In [ ]:
generator=Generator(64,3,128,norm_type="GroupNorm",final_activation="tanh")
discriminator=Discriminator(64,3,norm_type="GroupNorm",final_activation=None)
checkpoint=load_model("registry://wgan/WGAN-GP:1.0.0")
generator.load_state_dict(checkpoint["g_state_dict"])
discriminator.load_state_dict(checkpoint["d_state_dict"])

In [ ]:
device='cuda'
generator=generator.to(device)

In [ ]:
def random_sample(batch_size, z_dim, device):
        # input to the generator
        # z_dim channels, 1x1 pixels
        return torch.randn(batch_size,z_dim, 1, 1).to(device)

In [ ]:
def generate_images(nsamples):
        with torch.no_grad():
            noise = random_sample(256, 128, device)[:nsamples]
            fake_images = generator(noise)
        return fake_images
fake_images=generate_images(32)
grid = make_grid(fake_images, nrow=8, normalize=True)

In [ ]:
def recover_image(img):
        # PIL expects the image to be of shape (H,W,C)
        # in PyTorch it's (C,H,W)

        img=img.cpu().numpy().transpose(1, 2,0)*255
        return Image.fromarray(img.astype(np.uint8))
grid=recover_image(grid)
plt.imshow(grid)